In [51]:
from imdb_fasttext import lazy_load_imdb_data, fastText_keras
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.optim import SGD # Adam doesn't currently support autograd with embedding layers

In [3]:
(x_train, y_train), (x_test, y_test) = lazy_load_imdb_data()

Lazy load successful


In [6]:
fastText_keras(max_features=20000, embedding_dims=50, maxlen=400).summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 1,000,051
Trainable params: 1,000,051
Non-trainable params: 0
_________________________________________________________________


In [65]:
class TorchFastText(nn.Module):
    
    def __init__(self, max_features, embedding_dims, maxlen):
        super(TorchFastText, self).__init__()
        self.max_features = max_features
        self.embedding_dims = embedding_dims
        self.maxlen = maxlen
        
        self.embeds = nn.Embedding(max_features, embedding_dims)
        self.average = nn.AvgPool1d(kernel_size=(self.max_features, ) )
        self.dense = nn.Linear(self.embedding_dims, 1)
        
    def forward(self, x):
        embedded = self.embeds(Variable(x))
        averaged = self.average(embedded)
        z = self.dense(averaged)
        predicted = nn.Sigmoid(z)
        return predicted
    
model = TorchFastText(max_features, embedding_dims, maxlen)    

In [36]:
N = 32
max_features = 20000
embedding_dims = 50
maxlen = 400

In [53]:
binary_loss = nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.01)

In [48]:
from torch.utils.data import Dataset, DataLoader

class MyData(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __len__(self):
        return len(y)
    
    def __getitem__(self, idx):
        y_i = self.y[idx]
        x_i = self.x[idx]
        
        return {"x":x_i, "y":y_i}
    

In [49]:
training_data = MyData(x_train, y_train)
testing_data = MyData(x_test, y_test)

training_loader = DataLoader(training_data, batch_size=N)
testing_loader = DataLoader(testing_data, batch_size=N)

In [66]:
for batch in training_loader:
    # Get the inputs and wrap as Variables
    batch_x = torch.LongTensor(batch["x"].tolist())
    batch_y = Variable(batch["y"])
    
    # zero the parameter gradients
    optimizer.zero_grad()
    
    print(batch_x)

    # forward + backward + optimize
    outputs = model(batch_x)
    print("outputs: ")
    print(outputs)
    loss = binary_loss(outputs, batch_y)
    loss.backward()
    optimizer.step()
    break
    
    
    


     0      0      0  ...      19    178     32
     0      0      0  ...      16    145     95
     0      0      0  ...       7    129    113
        ...            ⋱           ...         
     0      0      0  ...       2   2366  17552
     0      0      0  ...    3931     78     22
    41    336     11  ...     209      6  10280
[torch.LongTensor of size 32x400]



RuntimeError: Given input size: (400x50x1). Calculated output size: (400x0x1). Output size is too small at /opt/conda/conda-bld/pytorch_1512386481460/work/torch/lib/THNN/generic/SpatialAveragePooling.c:64